# Caret-Separated Text

Caret-Separated Text (or CST) is a key-value pair format represented by numbers as keys and the value is the string enclosed between carets (^) that contains the translation. Any text which is not enclosed with carets is considered a comment and ignored.

## CST.NET

CST.NET uses .NET's built-in indexing extension function to accomplish locating of each respective key. As a consequence, it does not matter what you use for keys. I added an additional normalizion to the pipeline that converts the document's line endings to the system's, in order to prevent crashes.

In [1]:
using System.IO;

In [1]:
public static class CST
{
    public static string Parse(string cst, int key, params string[] args)
    {
        var entries = NormalizeEntries(cst);
        return GetEntry(entries, $"{key}", args);
    }

    public static string Parse(string cst, string key, params string[] args)
    {
        var entries = NormalizeEntries(cst);
        return GetEntry(entries, key, args);
    }

    static IEnumerable<string> NormalizeEntries(string cst)
    {
        var lineBreaks = new string[] { "^\u000A", "^\u000D", "^\u000A" };

        foreach (var line in lineBreaks)
        {
            var eol = Environment.NewLine; // System's line break

            // If the new line matches the system's, do nothing
            if (line.Contains(eol))
                continue;

            cst.Replace(line, eol);
        }

        return cst.Split(lineBreaks, StringSplitOptions.RemoveEmptyEntries);

    }

    // TODO: support argument parameters
    static string GetEntry(IEnumerable<string> entries, string key, params string[] args)
    {
        var translation = "[ENTRY NOT FOUND]";

        // Search through array
        foreach (var entry in entries)
        {
            // Locate index, trim carets and return translation
            if (!entry.StartsWith(key))
                continue;
                
            const char caret = '^';
            // const char token = '%';

            var startIndex = entry.IndexOf(caret.ToString(),
                StringComparison.OrdinalIgnoreCase);

            var line = entry.Substring(startIndex);

            /*foreach (var arg in args)
                Regex.Replace(line, $"%[1-100]", arg);*/

            translation = line.Trim(caret);
        }

        return translation;
    }
}

In [1]:
var v1Path = Path.Combine(Environment.CurrentDirectory, "data", "v1.cst");
var v1File = File.ReadAllText(v1Path);
var multiLine = CST.Parse(v1File, 1);
var singleLine = CST.Parse(v1File, 2);
Console.WriteLine($"Single line:{Environment.NewLine}{singleLine}");
Console.WriteLine($"Multiline:{Environment.NewLine}{multiLine}");

Single line:
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nunc pharetra nunc nec erat finibus efficitur. Duis non ullamcorper purus. Donec sit amet ultricies sapien. Sed lacinia sem eu nunc gravida, vitae tincidunt eros tempus. Quisque nibh est, tempus sit amet purus at, facilisis cursus ante. Nam nisi purus, vehicula sed magna ac, lobortis aliquam urna. Sed condimentum, felis a placerat tincidunt, est augue pulvinar turpis, eu dictum leo diam quis mi. Nulla non efficitur neque, sed efficitur orci. Aliquam quis libero consequat, convallis tortor sit amet, varius orci.

Orci varius natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Nunc vel dictum eros, vitae mattis risus. Curabitur eget nisi interdum, euismod nisl in, fermentum turpis. Morbi a feugiat lacus. Duis ligula felis, commodo quis sodales ac, congue sit amet tortor. Sed vulputate, velit id interdum convallis, purus nisl interdum lorem, sit amet aliquam lacus sapien ac neque. Proin sit amet u

Multiline:
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Proin ac dictum orci, at tincidunt nulla. Donec aliquet, eros non interdum posuere, ipsum sapien molestie nunc, nec facilisis libero ipsum et risus. In sed lorem vel ipsum placerat viverra.


In [1]:
var v2Path = Path.Combine(Environment.CurrentDirectory, "data", "v2.cst");
var v2File = File.ReadAllText(v2Path);
var multiLineV2 = CST.Parse(v2File, "Multiline");
var singleLineV2 = CST.Parse(v2File, "Singleline");
Console.WriteLine($"Single line v2:{Environment.NewLine}{singleLineV2}");
Console.WriteLine($"Multiline v2:{Environment.NewLine}{multiLineV2}");

Single line v2:
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed ultricies nulla eu tortor mattis, dictum posuere lacus ornare. Maecenas a massa in ligula finibus luctus eu vitae nibh. Proin imperdiet dapibus mauris quis placerat.


Multiline v2:
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nunc gravida nunc non justo pretium consectetur. Sed tempus libero ac ligula aliquam elementum. Duis vitae interdum leo. Sed semper nulla %1 a lectus dictum dictum. Ut mattis eu tortor in bibendum. Integer mattis tincidunt aliquet. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia Curae; Fusce quis orci nisl.

Quisque vehicula, nisi ut scelerisque sodales, nisi ipsum sodales ipsum, in rutrum tellus lacus sed nibh. Etiam mauris velit, elementum sed placerat et, elementum et tellus. Duis vitae elit fermentum, viverra lorem in, lobortis elit. Maecenas eget nibh et lectus auctor dignissim.
